In [32]:
import pandas as pd

df = pd.read_csv('Alzheimer.csv')

df.head()

,Unnamed: 0,relation,display_relation,x_index,x_id,x_type,x_name,x_source,y_index,y_id,y_type,y_name,y_source
0,3093313,disease_phenotype_positive,phenotype present,28310,7433,disease,dementia/parkinsonism with non-Alzheimer amylo...,MONDO,23153,1300,effect/phenotype,Parkinsonism,HPO
1,3093314,disease_phenotype_positive,phenotype present,28310,7433,disease,dementia/parkinsonism with non-Alzheimer amylo...,MONDO,22442,726,effect/phenotype,Dementia,HPO
2,3093315,disease_phenotype_positive,phenotype present,28310,7433,disease,dementia/parkinsonism with non-Alzheimer amylo...,MONDO,89049,11960,effect/phenotype,Substantia nigra gliosis,HPO
3,3093316,disease_phenotype_positive,phenotype present,28310,7433,disease,dementia/parkinsonism with non-Alzheimer amylo...,MONDO,22759,6,effect/phenotype,Autosomal dominant inheritance,HPO
4,3101044,disease_phenotype_positive,phenotype present,28780,11913_7088_11743_12153_7089_11194_12321_15140_...,disease,Alzheimer disease,MONDO_grouped,22999,2015,effect/phenotype,Dysphagia,HPO


In [31]:
from configparser import ConfigParser
from neo4j import GraphDatabase
import pandas as pd

def connect_neo4j():
    config = ConfigParser()
    config.read('config')

    username = config.get('Neo4j', 'username')
    password = config.get('Neo4j', 'password')
    host = config.get('Neo4j', 'host')

    driver = GraphDatabase.driver(host, auth=(username, password))

    return driver

def excute_query(query, parameters=''):
    with driver.session() as session:
        return session.run(query, parameters)


def insert_sample_data(driver):
    name = 'Tom Hanks'
    query = f'''CREATE (p:Person {{name: $name, born: 1956}})
                CREATE (m:Movie {{title: 'Forrest Gump', released: 1994}})
                CREATE (p)-[r:ACTED_IN {{roles: ['Forrest']}}]->(m)
                RETURN p, m, r'''
    parameters = { 'name': name }
    excute_query(query, parameters)


def insert_data(driver):
    df = pd.read_csv('Alzheimer.csv')
    # df = pd.read_csv('kg.csv')

    arr = df[['x_type', 'y_type', 'relation']].values
    unique_pairs = set(tuple(pair) for pair in arr)
    len(unique_pairs)

    for types in unique_pairs:

        mask = (df['x_type'] == types[0]) & (df['y_type'] == types[1]) & (df['relation'] == types[2])
        temp_df = df[mask]
        temp_df = temp_df.replace('/', '_', regex=True)
        
        replaced_types = tuple(s.replace('/', '_') for s in types)
        records = temp_df.to_dict('records')

        query = f'''
        UNWIND $records AS record
        MERGE (x:{replaced_types[0]} {{id: record.x_id, name: record.x_name, type: record.x_type, source: record.x_source}})
        MERGE (y:{replaced_types[1]} {{id: record.y_id, name: record.y_name, type: record.y_type, source: record.y_source}})
        MERGE (x)-[r:{replaced_types[2]} {{relation: record.relation, display_relation: record.display_relation}}]->(y)
        '''
    
        excute_query(query, parameters={'records': records})


def empty_database(driver):
    query = '''MATCH (n)
               DETACH DELETE n'''
    excute_query(query)

def list_record(driver):
    query = '''match (n) return (n)'''
    with driver.session() as session:
        records = session.run(query)

        for record in records:
            print(record.keys, record.values)

driver = connect_neo4j()


In [23]:
# clear database
empty_database(driver)

In [26]:
# insert_sample_data(driver)
insert_data(driver)

In [27]:
# list record
list_record(driver)


<bound method Record.keys of <Record n=<Node element_id='273' labels=frozenset({'disease'}) properties={'name': 'Alzheimer disease', 'id': '11913_7088_11743_12153_7089_11194_12321_15140_4975_11561_11647_11777_12344_12609_12630_12631_14265_10422_14036_14316_100088', 'source': 'MONDO_grouped', 'type': 'disease'}>>> <bound method Record.values of <Record n=<Node element_id='273' labels=frozenset({'disease'}) properties={'name': 'Alzheimer disease', 'id': '11913_7088_11743_12153_7089_11194_12321_15140_4975_11561_11647_11777_12344_12609_12630_12631_14265_10422_14036_14316_100088', 'source': 'MONDO_grouped', 'type': 'disease'}>>>
<bound method Record.keys of <Record n=<Node element_id='274' labels=frozenset({'exposure'}) properties={'name': 'Aluminum', 'id': 'D000535', 'source': 'CTD', 'type': 'exposure'}>>> <bound method Record.values of <Record n=<Node element_id='274' labels=frozenset({'exposure'}) properties={'name': 'Aluminum', 'id': 'D000535', 'source': 'CTD', 'type': 'exposure'}>>>
<b